In [1]:
import pandas as pd
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
)
from codebase.plot import plot_density, plot_histogram, plot_line, get_post_df
from codebase.ibis import post_process_sign, exp_and_normalise
import altair as alt
from codebase.classes_data import Data
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from pdb import set_trace
from scipy.special import logsumexp

alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

In [2]:
def get_model_evidence(log_dir):
    particles = load_obj('particles', log_dir)
    log_lklhds = load_obj('log_lklhds', log_dir)[30:]
    model_evidence = np.empty_like(log_lklhds)
    for i in range(log_lklhds.shape[0]):
        model_evidence[i] = np.exp(logsumexp(log_lklhds[:(i+1)]))
    return model_evidence

def plot_bf(df, width=500, height=200):
    return alt.Chart(df.melt(id_vars = 'idx')).mark_line(
        strokeWidth = 4,
        opacity=0.9
        ).encode(
        alt.X('idx:Q', title=None),
        alt.Y('value:Q'),
        alt.Color('variable')
        ).properties(
        width=width, height=height
        ).configure_axis(
        labelFontSize=14,
        titleFontSize=16).configure_legend(
        titleFontSize=16,
        labelFontSize=14,
        symbolStrokeWidth=10
        ).interactive()

### Particle Size = 100

In [3]:
data = load_obj('data','log/fabian_sim/20210829_134725_sim_m7/')
data.raw_data['beta']

array([[1. , 0. ],
       [0.8, 0.3],
       [0.8, 0. ],
       [0. , 1. ],
       [0.3, 0.8],
       [0.3, 0.8]])

In [5]:
model_evidence = get_model_evidence('log/fabian_sim/20210829_134725_sim_m7/')
bf = pd.DataFrame(model_evidence, columns=['EZ'])
bf['idx'] = np.arange(bf.shape[0])+30
bf['AZ'] = get_model_evidence('log/fabian_sim/20210829_134729_sim_m5/')
bf['EFA'] = get_model_evidence('log/fabian_sim/20210829_134730_sim_m9/')
bf['EFA-C'] = get_model_evidence('log/fabian_sim/20210829_134734_sim_m10/')
plot_bf(bf, 800, 400)

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


alt.Chart(...)

### Got 100 people from the Big5 data set

In [14]:
model_evidence = get_model_evidence('log/fabian_real_muthen/20210829_201308_smallbig5_m11/')
bf = pd.DataFrame(model_evidence, columns=['EZ'])
bf['idx'] = np.arange(bf.shape[0])
bf['AZ'] = get_model_evidence('log/fabian_real_muthen/20210829_201302_smallbig5_m6/')

bf['EFA'] = get_model_evidence('log/fabian_real_muthen/20210829_201305_smallbig5_m9/')
bf['EFA-C'] = get_model_evidence('log/fabian_real_muthen/20210829_201450_smallbig5_m10/')
plot_bf(bf, 800, 400)

alt.Chart(...)

In [15]:
bf['AZ/EZ'] = bf['AZ']/bf['EZ']
bf['AZ/EFA'] = bf['AZ']/bf['EFA']
bf['AZ/EFA-C'] = bf['AZ']/bf['EFA-C']
bf.drop(['AZ', 'EZ', 'EFA', 'EFA-C'], axis=1, inplace=True)
plot_bf(bf, 800, 400)

alt.Chart(...)

In [9]:
model_evidence = get_model_evidence('log/fabian_real_muthen/20210829_201302_smallbig5_m6/')/get_model_evidence('log/fabian_real_muthen/20210829_201308_smallbig5_m11/')
bf = pd.DataFrame(model_evidence, columns=['AZ/EZ'])
bf['idx'] = np.arange(bf.shape[0])
# bf['EFA'] = get_model_evidence('log/CL_data/20210726_213814_efa/')
# bf['EFA-C'] = get_model_evidence('log/CL_data/20210726_214548_efa_c/')
plot_bf(bf, 800, 400)

alt.Chart(...)